In [1]:
# --- 라이브러리 설치 ---
# 필수 라이브러리 설치
# pip가 Colab 환경에 맞는 호환 버전을 찾도록 함
# --- 라이브러리 설치 (2단계) ---

# 1단계: NumPy 버전 고정 (< 2)
!pip install "numpy<2" -q
print("1단계: NumPy < 2 설치 완료.")

# 2단계: 나머지 라이브러리 설치 (torch, torchvision, torchaudio 포함)
# pip가 NumPy 1.x 환경에 맞는 호환 버전을 찾도록 함
!pip install transformers torch torchvision torchaudio pandas biopython scikit-learn pyarrow -q --upgrade
print("2단계: 주요 라이브러리 설치/업데이트 완료.")

# --- Google Drive 마운트 ---
from google.colab import drive
import os

try:
    drive.mount('/content/drive')
    print("Google Drive 마운트 성공.")
except Exception as e:
    print(f"Google Drive 마운트 중 오류 발생: {e}")
    # 필요시 오류 처리

# --- 기본 경로 설정 (Colab 환경) ---
# Google Drive 내 작업 디렉토리 경로 (본인 환경에 맞게 수정)
base_dir = '/content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber'
if not os.path.exists(base_dir):
    print(f"경고: 기본 디렉토리를 찾을 수 없습니다: {base_dir}")
    print("Google Drive 경로를 확인하거나 디렉토리를 생성해주세요.")
    # os.makedirs(base_dir) # 필요시 디렉토리 생성

# 중간 결과 저장 경로 설정
feature_dir = os.path.join(base_dir, '2_feature_engineered')
split_dir = os.path.join(base_dir, '3_split_data')
os.makedirs(feature_dir, exist_ok=True)
os.makedirs(split_dir, exist_ok=True)

print(f"기본 디렉토리: {base_dir}")
print(f"특성 저장 디렉토리: {feature_dir}")
print(f"분할 데이터 저장 디렉토리: {split_dir}")

1단계: NumPy < 2 설치 완료.
2단계: 주요 라이브러리 설치/업데이트 완료.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 마운트 성공.
기본 디렉토리: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber
특성 저장 디렉토리: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/2_feature_engineered
분할 데이터 저장 디렉토리: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/3_split_data


In [2]:
# 임포트 테스트 셀 : 위 실행후 런타임 재시작 해야
try:
    import pandas as pd
    import numpy as np
    from Bio import SeqIO
    from Bio.Seq import Seq
    from Bio.SeqRecord import SeqRecord
    from sklearn.model_selection import GroupShuffleSplit
    from transformers import AutoTokenizer, AutoModel
    import torch
    import os
    import re
    import time
    print("--- 필수 라이브러리 임포트 성공 ---")
    print(f"NumPy version: {np.__version__}")
    print(f"Pandas version: {pd.__version__}")
    # 필요시 다른 라이브러리 버전도 확인
    # import transformers
    # print(f"Transformers version: {transformers.__version__}")
    # import sklearn
    # print(f"Scikit-learn version: {sklearn.__version__}")
except Exception as e:
    print(f"오류: 라이브러리 임포트 중 문제 발생: {e}")
    # 오류 발생 시 여기서 멈추고 원인 파악

--- 필수 라이브러리 임포트 성공 ---
NumPy version: 1.26.4
Pandas version: 2.2.3


In [3]:
import os
import time
import re # 클러스터 파일 파싱용
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from sklearn.model_selection import GroupShuffleSplit
from transformers import AutoTokenizer, AutoModel
import torch

# --- GPU 상태 확인 ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'GPU 사용 가능: {torch.cuda.get_device_name(0)}')
    # 사용 가능한 총 메모리와 현재 할당된 메모리 확인 (Bytes 단위)
    total_mem = torch.cuda.get_device_properties(0).total_memory
    free_mem = total_mem - torch.cuda.memory_allocated(0)
    print(f'GPU 총 메모리: {total_mem / (1024**3):.2f} GB')
    print(f'GPU 사용 가능 메모리 (추정): {free_mem / (1024**3):.2f} GB')
    # 3B 모델은 최소 12-16GB 이상의 VRAM 권장
    if free_mem / (1024**3) < 12:
         print("경고: 사용 가능한 GPU 메모리가 12GB 미만입니다. 3B 모델 실행 시 메모리 부족 오류가 발생할 수 있습니다.")
         print("모델 크기를 줄이거나 배치 크기를 매우 작게 조절해야 할 수 있습니다.")
else:
    device = torch.device("cpu")
    print('GPU 사용 불가, CPU 사용.')
    print("경고: CPU 사용 시 임베딩 생성 속도가 매우 느립니다.")

GPU 사용 불가, CPU 사용.
경고: CPU 사용 시 임베딩 생성 속도가 매우 느립니다.


In [4]:
# --- 데이터 경로 설정 ---
metadata_path = os.path.join(base_dir, '1_preprocessed', 'final_dataset', 'antibody_metadata_abnumber.parquet')
fasta_path = os.path.join(base_dir, '1_preprocessed', 'final_dataset', 'sequences', 'final_antibody_seqs_abnumber.fasta')

# --- 메타데이터 로드 ---
print(f"메타데이터 로딩 중: {metadata_path}")
try:
    metadata_df = pd.read_parquet(metadata_path)
    print(f"메타데이터 로드 완료: {len(metadata_df)} 항목")
    # 필수 컬럼 확인
    required_cols = ['entry_id', 'vh_sequence', 'vl_sequence', 'vh_cdr3']
    if not all(col in metadata_df.columns for col in required_cols):
        missing_cols = [col for col in required_cols if col not in metadata_df.columns]
        print(f"오류: 메타데이터에 필수 컬럼이 없습니다 - {missing_cols}")
        # 필요한 조치 수행 (예: 스크립트 중단)
        raise ValueError("필수 메타데이터 컬럼 누락")
except FileNotFoundError:
    print(f"오류: 메타데이터 파일을 찾을 수 없습니다 - {metadata_path}")
    # 필요한 조치 수행
    raise
except Exception as e:
    print(f"메타데이터 로드 중 오류 발생: {e}")
    raise

# --- 서열 데이터 준비 ---
# 메타데이터에 서열 정보가 있으므로, 이를 사용
vh_sequences = metadata_df['vh_sequence'].tolist()
vl_sequences = metadata_df['vl_sequence'].tolist()
print(f"VH/VL 서열 리스트 준비 완료: 각 {len(vh_sequences)} 개")

# FASTA 파일 로드는 검증용으로 사용 가능 (선택 사항)
# sequences = {}
# try:
#     for record in SeqIO.parse(fasta_path, "fasta"):
#         entry_id, chain_type = record.id.rsplit('_', 1)
#         if entry_id not in sequences:
#             sequences[entry_id] = {}
#         sequences[entry_id][chain_type] = str(record.seq)
#     print(f"FASTA 파일 로드 완료 (검증용): {len(sequences)} 항목")
# except FileNotFoundError:
#     print(f"경고: FASTA 파일을 찾을 수 없습니다 - {fasta_path}")
# except Exception as e:
#     print(f"FASTA 파일 로드 중 오류 발생: {e}")

메타데이터 로딩 중: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/1_preprocessed/final_dataset/antibody_metadata_abnumber.parquet
메타데이터 로드 완료: 10594 항목
VH/VL 서열 리스트 준비 완료: 각 10594 개


In [5]:
# --- 모델 선택 ---
# 옵션 1: 650M 모델 (Colab T4 GPU에서 실행 가능성 높음)
# model_name = "facebook/esm2_t33_650M_UR50D"
# batch_size_auto = 16 # T4 GPU 기준 추천 배치 크기 (메모리 부족 시 8로 줄임)

# 옵션 2: 3B 모델 (Colab T4에서 메모리 부족 가능성 매우 높음, A100 등 고사양 GPU 필요)
model_name = "facebook/esm2_t36_3B_UR50D"
batch_size_auto = 4 # 3B 모델 + T4 GPU 사용 시 매우 작은 배치 크기 필요 (그래도 OOM 가능성 있음)

# 옵션 3: 항체 특화 모델 (항체 특화 작업에 유리할 수 있음)
# model_name = "ación/AntiBERTy" # 정확한 모델 이름을 Hugging Face Hub에서 확인 필요

print(f"사용할 모델: {model_name}")
print(f"자동 설정 배치 크기: {batch_size_auto} (GPU 메모리 따라 조절 필요)")

# --- 모델 및 토크나이저 로드 ---
print("모델 및 토크나이저 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # allow_remote_code=True 는 신뢰할 수 있는 소스에서만 사용
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True) # 일부 모델은 이 옵션 필요할 수 있음
    model.to(device)
    model.eval() # 평가 모드 설정
    print("모델 및 토크나이저 로드 완료.")
except Exception as e:
    print(f"모델 로딩 중 오류 발생: {e}")
    # GPU 메모리 부족(OOM) 오류 발생 시 모델 크기나 배치 크기 재고려 필요
    raise

# --- 개선된 임베딩 생성 함수 ---
def get_embeddings(sequence_list, batch_size=8, model=model, tokenizer=tokenizer, device=device, max_len=1024, progress_interval=10):
    """
    주어진 서열 리스트로부터 pLM 임베딩을 생성합니다. (평균 풀링 방식, 패딩 제외)

    Args:
        sequence_list (list): 아미노산 서열 문자열 리스트.
        batch_size (int): 한 번에 처리할 서열 수. GPU 메모리에 따라 조절.
        model: 사전 학습된 pLM 모델 객체.
        tokenizer: 해당 모델의 토크나이저 객체.
        device: 사용할 디바이스 ('cuda' 또는 'cpu').
        max_len (int): 토크나이저의 최대 입력 길이.
        progress_interval (int): 로그 출력 빈도 (배치 단위).

    Returns:
        numpy.ndarray: 각 서열에 대한 임베딩 벡터 배열 (N x embedding_dim).
    """
    all_embeddings = []
    num_sequences = len(sequence_list)
    num_batches = (num_sequences + batch_size - 1) // batch_size
    start_time_total = time.time()
    print(f"총 서열 수: {num_sequences}, 배치 크기: {batch_size}, 총 배치 수: {num_batches}")

    # 모델을 평가 모드로 설정
    model.eval()

    for i in range(num_batches):
        start_time_batch = time.time()
        # 배치 범위 계산 (리스트 인덱스 초과 방지)
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_sequences)
        batch_sequences = sequence_list[start_idx:end_idx]

        if not batch_sequences: # 혹시 모를 빈 배치 처리
            continue

        # 토크나이징 (패딩 및 트렁케이션 포함, PyTorch 텐서로 반환)
        try:
            inputs = tokenizer(
                batch_sequences,
                return_tensors="pt",
                padding="max_length", # max_length까지 패딩
                truncation=True,      # max_length 초과 시 잘라냄
                max_length=max_len    # 최대 길이 설정 (ESM 권장: 1024)
            ).to(device)
        except Exception as e:
            print(f"오류: 배치 {i+1} 토크나이징 중 오류 발생: {e}")
            print(f"오류 발생 서열 (일부): {batch_sequences[:2]}")
            # 오류 발생 시 빈 리스트나 None 반환 등 처리 필요
            return None # 또는 빈 배열 np.array([])

        # 그래디언트 계산 비활성화 (메모리 절약 및 속도 향상)
        with torch.no_grad():
            try:
                # 모델 실행
                outputs = model(**inputs)

                # 마지막 은닉 상태 (batch_size, sequence_length, hidden_size)
                last_hidden = outputs.last_hidden_state

                # Attention mask (패딩 토큰은 0, 실제 토큰은 1)
                attention_mask = inputs['attention_mask']

                # 패딩 토큰 제외하고 실제 토큰의 임베딩만 평균 계산
                # 1. 마스크 확장: (batch, seq_len) -> (batch, seq_len, hidden_size)
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
                # 2. 실제 토큰 임베딩 합계 계산 (마스크된 부분은 0이 됨)
                sum_embeddings = torch.sum(last_hidden * input_mask_expanded, 1)
                # 3. 실제 토큰 수 계산 (0으로 나누는 것 방지)
                sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
                # 4. 평균 임베딩 계산
                mean_embeddings = sum_embeddings / sum_mask

                # 결과를 CPU로 이동하고 NumPy 배열로 변환하여 리스트에 추가
                all_embeddings.extend(mean_embeddings.cpu().numpy())

            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"오류: 배치 {i+1} 처리 중 GPU 메모리 부족(OOM) 발생!")
                    print("배치 크기를 줄여서 다시 시도해보세요.")
                    # OOM 발생 시 None 반환하여 상위에서 처리하도록 함
                    return None
                else:
                    print(f"오류: 배치 {i+1} 모델 실행 중 런타임 오류 발생: {e}")
                    return None # 또는 빈 배열
            except Exception as e:
                 print(f"오류: 배치 {i+1} 처리 중 예외 발생: {e}")
                 return None # 또는 빈 배열

        end_time_batch = time.time()
        # 지정된 간격마다 진행 상황 출력
        if (i + 1) % progress_interval == 0 or (i + 1) == num_batches:
            print(f"  배치 {i+1}/{num_batches} 처리 완료. (배치 당 소요 시간: {end_time_batch - start_time_batch:.2f} 초)")

    end_time_total = time.time()
    print(f"총 임베딩 생성 시간: {end_time_total - start_time_total:.2f} 초")

    # 최종 결과를 NumPy 배열로 변환하여 반환
    return np.array(all_embeddings)

사용할 모델: facebook/esm2_t36_3B_UR50D
자동 설정 배치 크기: 4 (GPU 메모리 따라 조절 필요)
모델 및 토크나이저 로딩 중...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 및 토크나이저 로드 완료.


In [6]:
# --- 임베딩 생성 실행 ---
# batch_size는 앞에서 설정한 batch_size_auto 사용 또는 직접 지정
# Colab T4 + 3B 모델 사용 시 매우 작은 값(예: 2, 4) 필요, OOM 발생 시 더 줄여야 함
effective_batch_size = batch_size_auto # 또는 4, 8 등 직접 지정

print("\n--- VH 서열 임베딩 생성 시작 ---")
vh_embeddings = get_embeddings(vh_sequences, batch_size=effective_batch_size)

# OOM 등으로 인해 None 반환 시 처리
if vh_embeddings is None:
    print("오류: VH 임베딩 생성 실패. 배치 크기 또는 모델을 확인하세요.")
    # 필요시 여기서 중단
else:
    print("\n--- VL 서열 임베딩 생성 시작 ---")
    vl_embeddings = get_embeddings(vl_sequences, batch_size=effective_batch_size)

    if vl_embeddings is None:
         print("오류: VL 임베딩 생성 실패. 배치 크기 또는 모델을 확인하세요.")
    else:
        # --- 결과 저장 ---
        # 모델 이름과 크기를 파일명에 포함하여 구분 용이하게 함
        model_name_safe = model_name.split('/')[-1].replace('-', '_') # 파일명으로 사용 가능하게 변환
        vh_output_filename = f'vh_embeddings_{model_name_safe}.npy'
        vl_output_filename = f'vl_embeddings_{model_name_safe}.npy'
        vh_output_path = os.path.join(feature_dir, vh_output_filename)
        vl_output_path = os.path.join(feature_dir, vl_output_filename)

        try:
            print(f"\nVH 임베딩 저장 중: {vh_output_path}")
            np.save(vh_output_path, vh_embeddings)
            print(f"VL 임베딩 저장 중: {vl_output_path}")
            np.save(vl_output_path, vl_embeddings)

            print(f"\n임베딩 저장 완료:")
            print(f"VH 임베딩 shape: {vh_embeddings.shape}")
            print(f"VL 임베딩 shape: {vl_embeddings.shape}")

            # 임베딩 경로를 메타데이터에 추가 (선택 사항)
            # metadata_df['vh_embedding_path'] = vh_output_path
            # metadata_df['vl_embedding_path'] = vl_output_path
            # metadata_output_path = os.path.join(feature_dir, 'metadata_with_embedding_paths.parquet')
            # metadata_df.to_parquet(metadata_output_path)
            # print(f"임베딩 경로 포함된 메타데이터 저장: {metadata_output_path}")

        except Exception as e:
            print(f"오류: 임베딩 저장 중 오류 발생: {e}")


--- VH 서열 임베딩 생성 시작 ---
총 서열 수: 10594, 배치 크기: 4, 총 배치 수: 2649
  배치 10/2649 처리 완료. (배치 당 소요 시간: 138.10 초)
  배치 20/2649 처리 완료. (배치 당 소요 시간: 176.74 초)
  배치 30/2649 처리 완료. (배치 당 소요 시간: 182.11 초)


KeyboardInterrupt: 

In [ ]:
# --- CD-HIT 설치 (Colab 환경) ---
# 필요한 빌드 도구 설치
print("CD-HIT 설치 시작 (빌드 도구 설치 및 컴파일)...")
!sudo apt-get update -qq > /dev/null
!sudo apt-get install -y build-essential -qq > /dev/null

# CD-HIT 소스 다운로드 및 압축 해제
!wget https://github.com/weizhongli/cdhit/releases/download/V4.8.1/cd-hit-v4.8.1-2019-0228.tar.gz -q
!tar -xzf cd-hit-v4.8.1-2019-0228.tar.gz

# CD-HIT 컴파일 (오류 발생 시 로그 확인 필요)
print("CD-HIT 컴파일 중...")
compile_log = !cd cd-hit-v4.8.1-2019-0228 && make
if "error" in "".join(compile_log).lower():
     print("오류: CD-HIT 컴파일 실패!")
     print("\n".join(compile_log))
else:
     print("CD-HIT 컴파일 성공.")
     # 실행 파일 경로 설정
     cd_hit_executable = '/content/cd-hit-v4.8.1-2019-0228/cd-hit'
     # 실행 권한 부여 (필요한 경우)
     !chmod +x {cd_hit_executable}
     # 버전 확인
     !{cd_hit_executable} -h | head -n 5

# 컴파일 실패 시 대안: 사전 컴파일된 바이너리 사용 시도 (제공 여부 확인 필요)
# 또는 웹 서버 이용 고려

In [ ]:
# --- CD-HIT 실행 준비 ---

# 1. 클러스터링 기준 서열 선택 및 FASTA 파일 생성
#    기준: VH CDR3 (가장 다양성이 높음). 다른 기준(예: VH 전체) 사용 가능.
#    이전 셀에서 metadata_df가 로드되었다고 가정.
print("\n--- CD-HIT 입력 FASTA 파일 생성 ---")
cdr3_sequences_for_cdhit = []
if 'vh_cdr3' in metadata_df.columns:
    for index, row in metadata_df.iterrows():
        entry_id = row['entry_id'] # 각 서열의 고유 ID
        cdr3_seq = str(row['vh_cdr3']) if pd.notna(row['vh_cdr3']) else ""
        if cdr3_seq and len(cdr3_seq) > 0: # 유효한 CDR3 서열만 포함
            # SeqRecord 생성 (FASTA 헤더 >entry_id)
            record = SeqRecord(Seq(cdr3_seq), id=str(entry_id), description="") # ID는 문자열이어야 함
            cdr3_sequences_for_cdhit.append(record)
        else:
            print(f"경고: Entry ID {entry_id}의 VH CDR3 서열이 유효하지 않아 제외됩니다.")
else:
    print("오류: 'vh_cdr3' 컬럼이 메타데이터에 없습니다.")
    # 오류 처리 또는 중단 필요
    raise ValueError("'vh_cdr3' 컬럼 누락")

# FASTA 파일 저장 경로 (Colab 내 임시 저장 또는 Drive)
# 작업 편의상 /content/ 에 저장 후 필요시 Drive로 복사
input_fasta_filename = 'vh_cdr3_sequences_for_cdhit.fasta'
input_fasta_path = f'/content/{input_fasta_filename}' # Colab 임시 경로

try:
    with open(input_fasta_path, "w") as output_handle:
        SeqIO.write(cdr3_sequences_for_cdhit, output_handle, "fasta")
    print(f"VH CDR3 서열 ({len(cdr3_sequences_for_cdhit)}개) FASTA 파일 저장 완료: {input_fasta_path}")
except Exception as e:
    print(f"FASTA 파일 저장 중 오류 발생: {e}")
    raise

# 2. 출력 파일 Prefix 설정 (결과 파일들이 저장될 기본 이름)
#    결과 파일 저장 위치: feature_dir 또는 split_dir 사용
output_prefix_base = 'vh_cdr3_clusters_90' # 예시: 90% 유사도 기준 클러스터
output_prefix = os.path.join(feature_dir, output_prefix_base) # 결과 파일 저장 경로
# 주의: Colab의 ! 명령어는 Google Drive 경로에 직접 쓰는 것이 불안정할 수 있음.
# /content/ 에 생성 후 Drive로 복사하는 것을 권장.
colab_output_prefix = f'/content/{output_prefix_base}'

# 3. CD-HIT 실행 옵션 설정
identity_threshold = 0.9 # 서열 유사도 임계값 (0.9 = 90%). 모델 검증 엄격성을 위해 0.8 등 더 낮출 수 있음.
word_size = 5 # n-gram 워드 크기. 짧은 CDR 서열에는 4 또는 5가 적합. (CD-HIT 문서 참조)
# 참고: CDR3 평균 길이가 약 16이므로, word_size=5는 적절한 시작점.

# 4. CD-HIT 실행 (Colab ! 명령어 사용)
#    실행 파일 경로: 이전 셀에서 설정한 cd_hit_executable 사용
command = f"{cd_hit_executable} -i {input_fasta_path} -o {colab_output_prefix} -c {identity_threshold} -n {word_size} -M 0 -T 0"
# -M 0: 메모리 제한 없음 (Colab 환경 고려)
# -T 0: 사용 가능한 모든 스레드 사용

print(f"\n--- CD-HIT 실행 (Colab) ---")
print(f"명령어: {command}")

try:
    # ! 명령어 실행 및 결과 저장
    cdhit_output = !{command}
    # 실행 결과 출력 (디버깅용)
    print("\n--- CD-HIT 실행 로그 ---")
    for line in cdhit_output:
        print(line)
    print("---------------------")

    # 출력 파일 존재 확인
    cluster_file_path_colab = f"{colab_output_prefix}.clstr"
    if os.path.exists(cluster_file_path_colab):
        print(f"CD-HIT 실행 성공. 클러스터 파일 생성됨: {cluster_file_path_colab}")
        # 생성된 파일을 Google Drive로 복사 (선택 사항)
        cluster_file_path_drive = f"{output_prefix}.clstr"
        !cp {cluster_file_path_colab} "{cluster_file_path_drive}" # 경로에 공백 있을 수 있으므로 따옴표 사용
        print(f"클러스터 파일을 Drive로 복사 완료: {cluster_file_path_drive}")
    else:
        print(f"오류: CD-HIT 실행은 되었으나 클러스터 파일({cluster_file_path_colab})이 생성되지 않았습니다.")
        print("CD-HIT 로그를 확인하여 원인을 파악하세요.")
        # 오류 처리

except Exception as e:
    print(f"CD-HIT 실행 중 예외 발생: {e}")
    # 오류 처리

In [ ]:
def parse_cdhit_clstr(clstr_file_path):
    """
    CD-HIT의 .clstr 파일을 파싱하여 각 서열 ID와 클러스터 ID를 매핑하는 딕셔너리를 반환합니다.

    Args:
        clstr_file_path (str): .clstr 파일의 경로.

    Returns:
        dict: {entry_id: cluster_id} 형태의 딕셔너리.
               파싱 실패 시 None 반환.
    """
    clusters = {}
    current_cluster_id = -1
    try:
        with open(clstr_file_path, 'r') as f:
            for line in f:
                if line.startswith('>Cluster'):
                    # 새 클러스터 시작, 클러스터 ID 추출 (예: >Cluster 0 -> 0)
                    try:
                        current_cluster_id = int(line.strip().split()[-1])
                    except (IndexError, ValueError):
                         print(f"경고: 클러스터 ID 파싱 오류 - {line.strip()}")
                         current_cluster_id = -1 # 오류 발생 시 유효하지 않은 ID로 설정
                elif line.strip():
                    # 클러스터 멤버 라인
                    # 예: 0   117aa, >SAbDab_8h9h_VH... *
                    match = re.search(r'>(\S+)\.\.\.', line) # FASTA 헤더 ID (entry_id) 추출
                    if match and current_cluster_id != -1:
                        entry_id = match.group(1)
                        clusters[entry_id] = current_cluster_id
                    else:
                        # FASTA 헤더에서 ID 추출 실패 시 경고
                         print(f"경고: 서열 ID 파싱 오류 또는 유효하지 않은 클러스터 ID - {line.strip()}")
        print(f"클러스터 파일 파싱 완료: 총 {len(set(clusters.values()))}개 클러스터, {len(clusters)}개 서열 매핑됨.")
        return clusters
    except FileNotFoundError:
        print(f"오류: 클러스터 파일을 찾을 수 없습니다 - {clstr_file_path}")
        return None
    except Exception as e:
        print(f"클러스터 파일 파싱 중 오류 발생: {e}")
        return None

In [ ]:
# --- 클러스터 파일 파싱 실행 ---
# 이전 셀에서 Drive로 복사된 클러스터 파일 경로 사용
cluster_file_path = f"{output_prefix}.clstr" # Drive 내 클러스터 파일 경로
print(f"\n--- 클러스터 파일 파싱 시작: {cluster_file_path} ---")
entry_to_cluster = parse_cdhit_clstr(cluster_file_path)

if entry_to_cluster is None:
    print("오류: 클러스터 정보 파싱 실패. 데이터 분할을 진행할 수 없습니다.")
    # 오류 처리 또는 중단
    raise ValueError("클러스터 파싱 실패")
else:
    # --- 그룹 정보 생성 ---
    # 메타데이터의 entry_id를 기준으로 클러스터 ID(그룹) 할당
    # CD-HIT에 포함되지 않은 서열(예: CDR3 없거나 짧음) 처리 필요
    groups = metadata_df['entry_id'].map(entry_to_cluster).fillna(-1).astype(int) # 클러스터 없으면 -1 할당
    metadata_df['cluster_id'] = groups # 메타데이터에 클러스터 ID 추가

    # 클러스터링되지 않은 데이터 수 확인
    num_unclustered = (groups == -1).sum()
    if num_unclustered > 0:
        print(f"경고: {num_unclustered}개의 서열이 클러스터링되지 않았습니다 (그룹 ID: -1). 이들은 별도로 처리될 수 있습니다.")

    # --- 데이터 분할 실행 (GroupShuffleSplit 사용) ---
    # 특성 데이터 로드 (이전에 생성된 임베딩 파일 경로 확인)
    try:
        model_name_safe = model_name.split('/')[-1].replace('-', '_')
        vh_embeddings_path = os.path.join(feature_dir, f'vh_embeddings_{model_name_safe}.npy')
        vl_embeddings_path = os.path.join(feature_dir, f'vl_embeddings_{model_name_safe}.npy')
        vh_embeddings = np.load(vh_embeddings_path)
        vl_embeddings = np.load(vl_embeddings_path)
        print(f"임베딩 데이터 로드 완료: {vh_embeddings_path}, {vl_embeddings_path}")
    except FileNotFoundError:
        print("오류: 저장된 임베딩 파일을 찾을 수 없습니다. 이전 단계를 확인하세요.")
        raise
    except Exception as e:
        print(f"임베딩 로드 중 오류 발생: {e}")
        raise

    # 데이터 인덱스 준비 (클러스터링된 데이터만 사용하거나, -1 그룹 처리 방식 결정 필요)
    # 여기서는 모든 데이터를 사용하되, -1 그룹은 각자 다른 그룹처럼 취급될 수 있음
    indices = np.arange(len(metadata_df))

    # 훈련/테스트 분할 (예: 80% 훈련, 20% 테스트)
    # n_splits=1: 한 번만 분할 수행
    # test_size=0.2: 테스트 세트 비율 20%
    # random_state: 재현성을 위한 시드 값
    gss_test = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    print("\n--- 훈련/테스트 데이터 분할 (GroupShuffleSplit) ---")
    train_val_idx, test_idx = next(gss_test.split(indices, groups=groups))

    # 훈련 세트를 다시 훈련/검증으로 분할 (예: 훈련 데이터 중 약 10~15% 검증)
    # test_size 계산 시 전체 데이터가 아닌 train_val_idx 기준으로 비율 설정 필요
    validation_split_ratio = 0.125 # 예: 80% 중 12.5% -> 전체의 10%
    gss_val = GroupShuffleSplit(n_splits=1, test_size=validation_split_ratio, random_state=42)
    print("--- 훈련/검증 데이터 분할 (GroupShuffleSplit) ---")
    train_idx, val_idx = next(gss_val.split(indices[train_val_idx], groups=groups[train_val_idx]))

    # 원본 데이터프레임 기준의 최종 인덱스 계산
    original_train_idx = train_val_idx[train_idx]
    original_val_idx = train_val_idx[val_idx]
    original_test_idx = test_idx # gss_test에서 반환된 인덱스 사용

    # --- 분할된 데이터 생성 ---
    # 임베딩 데이터 분할
    X_train_vh, X_val_vh, X_test_vh = vh_embeddings[original_train_idx], vh_embeddings[original_val_idx], vh_embeddings[original_test_idx]
    X_train_vl, X_val_vl, X_test_vl = vl_embeddings[original_train_idx], vl_embeddings[original_val_idx], vl_embeddings[original_test_idx]

    # 메타데이터 분할
    metadata_train = metadata_df.iloc[original_train_idx].copy() # SettingWithCopyWarning 방지
    metadata_val = metadata_df.iloc[original_val_idx].copy()
    metadata_test = metadata_df.iloc[original_test_idx].copy()

    print("\n--- 데이터 분할 결과 ---")
    print(f"훈련 세트 크기: {len(metadata_train)}")
    print(f"검증 세트 크기: {len(metadata_val)}")
    print(f"테스트 세트 크기: {len(metadata_test)}")
    print(f"총합: {len(metadata_train) + len(metadata_val) + len(metadata_test)} (원본: {len(metadata_df)})")

    # 분할 비율 확인 (근사치)
    total_size = len(metadata_df)
    print(f"분할 비율 (Train/Val/Test): {len(metadata_train)/total_size:.2f} / {len(metadata_val)/total_size:.2f} / {len(metadata_test)/total_size:.2f}")

    # --- 분할된 데이터 저장 ---
    print(f"\n--- 분할된 데이터 저장 시작 ({split_dir}) ---")
    try:
        # 임베딩 저장
        np.save(os.path.join(split_dir, 'X_train_vh.npy'), X_train_vh)
        np.save(os.path.join(split_dir, 'X_val_vh.npy'), X_val_vh)
        np.save(os.path.join(split_dir, 'X_test_vh.npy'), X_test_vh)
        np.save(os.path.join(split_dir, 'X_train_vl.npy'), X_train_vl)
        np.save(os.path.join(split_dir, 'X_val_vl.npy'), X_val_vl)
        np.save(os.path.join(split_dir, 'X_test_vl.npy'), X_test_vl)
        print("임베딩 데이터 (Train/Val/Test) 저장 완료.")

        # 메타데이터 저장
        metadata_train.to_parquet(os.path.join(split_dir, 'metadata_train.parquet'))
        metadata_val.to_parquet(os.path.join(split_dir, 'metadata_val.parquet'))
        metadata_test.to_parquet(os.path.join(split_dir, 'metadata_test.parquet'))
        print("메타데이터 (Train/Val/Test) 저장 완료.")

    except Exception as e:
        print(f"분할된 데이터 저장 중 오류 발생: {e}")